In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 1839, done.
remote: Counting objects: 100% (688/688), done.
remote: Compressing objects: 100% (230/230), done.
remote: Total 1839 (delta 545), reused 568 (delta 454), pack-reused 1151
Receiving objects: 100% (1839/1839), 158.50 MiB | 29.34 MiB/s, done.
Resolving deltas: 100% (1375/1375), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 21.3 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In functio

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps
from concurrent.futures import ThreadPoolExecutor

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/training-tuned-models/slim_bpr_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/__results__.html
/kaggle/input/training-tuned-models/slim_bpr_all.zip
/kaggle/input/training-tuned-models/__notebook__.ipynb
/kaggle/input/training-tuned-models/ials_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/ials_train_validation.zip
/kaggle/input/training-tuned-models/__output__.json
/kaggle/input/training-tuned-models/slim_bpr_train_validation.zip
/kaggle/input/training-tuned-models/ials_all.zip
/kaggle/input/training-tuned-models/custom.css
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/run_all_algorithms.py
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/cython_example_FunkSVD.py
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/LICENSE
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/.gitignore
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/rp3-tuning.ipynb
/kaggle/input/training-tuned-models/Recsys_Ch

In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender, SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.BaseRecommender import BaseRecommender

from Recommenders.Hybrid.LinearCombinationRecommender import LinearCombinationRecommender
from Data_manager.CrossValidationSplitter import CrossValidationSplitter

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_hybrid_linear_comb.db',
    'copy_prev_tuning_db': True,
    'copy_prev_best_params': True,
    'save_github': True
}

In [10]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [11]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [12]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

In [13]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_item.json', 'r') as params_file:
    params_item_knn = json.load(params_file)
params_item_knn

{'topK': 14,
 'shrink': 25,
 'similarity': 'tversky',
 'tversky_alpha': 0.18163095712123836,
 'tversky_beta': 1.8529812241698465}

In [14]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_RP3.json', 'r') as params_file:
    params_rp3 = json.load(params_file)
params_rp3

{'alpha': 0.26348679053501006, 'beta': 0.17536156577039744, 'topK': 27}

In [15]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_user.json', 'r') as params_file:
    params_user_knn = json.load(params_file)
params_user_knn

{'topK': 498,
 'shrink': 0,
 'similarity': 'asymmetric',
 'asymmetric_alpha': 0.39792525383867455}

In [16]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_alpha.json', 'r') as params_file:
    params_p3alpha = json.load(params_file)
params_p3alpha

{'normalize_similarity': 'True',
 'implicit': 'True',
 'alpha': 0.29354071531607107,
 'topK': 49}

In [17]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_hybrid_merging.json', 'r') as params_file:
    params_merging = json.load(params_file)
params_merging

{'alpha': 0.3375591188839621, 'topK': 86}

In [18]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/pure_svd_params.json', 'r') as params_file:
    params_pure_svd = json.load(params_file)
params_pure_svd

{'factors': 30}

In [19]:
# with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_puresvd_item.json', 'r') as params_file:
#     params_pure_svd_item = json.load(params_file)
params_pure_svd_item = {"topK": 43, "num_factors": 171}

In [20]:
shutil.copyfile(
    '/kaggle/input/training-tuned-models/ials_train_validation.zip', 
    '/kaggle/working/ials_train_validation.zip', 
)
shutil.copyfile(
    '/kaggle/input/training-tuned-models/ials_train_validation.zip', 
    '/kaggle/working/ials_all.zip', 
)

'/kaggle/working/ials_all.zip'

In [21]:
recommender_item = ItemKNNCFRecommender(URM_train)
recommender_item.fit(**params_item_knn)

recommender_user = UserKNNCFRecommender(URM_train)
recommender_user.fit(**params_user_knn)

recommender_rp3 = RP3betaRecommender(URM_train)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_train)
recommender_p3a.fit(**params_p3alpha)

recommender_pure_svd = PureSVDRecommender(URM_train)
recommender_pure_svd.fit(num_factors=params_pure_svd['factors'])

recommender_pure_svd_item = PureSVDItemRecommender(URM_train)
recommender_pure_svd_item.fit(**params_pure_svd_item)

recommender_nmf = NMFRecommender(URM_train)
recommender_nmf.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'nmf_train_validation.zip')

recommender_slim_bpr = SLIM_BPR_Python(URM_train)
recommender_slim_bpr.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_bpr_train_validation.zip')

recommender_slim = SLIMElasticNetRecommender(URM_train)
recommender_slim.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_elastic_train_validation.zip')

recommender_ease_r = EASE_R_Recommender(URM_train)
recommender_ease_r.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'ease_r_train_validation.zip')

recommender_ials = IALSRecommender(URM_train)
recommender_ials.load_model('/kaggle/working/', 'ials_train_validation.zip')

hybrid_similarity = ItemKNNSimilarityHybridRecommender(URM_train, recommender_item.W_sparse, recommender_rp3.W_sparse)
hybrid_similarity.fit(**params_merging)

ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 4572.18 column/sec. Elapsed time 4.89 sec
UserKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 13024 (100.0%), 4553.95 column/sec. Elapsed time 2.86 sec
RP3betaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2614.70 column/sec. Elapsed time 8.55 sec
P3alphaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
P3alphaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 2813.19 column/sec. Elapsed time 7.94 sec
PureSVDRecommender: URM Detected 627 ( 4.8%) users with no inter

In [22]:
weights = [0.4, 0.4, 0.2]
linear_combinator = LinearCombinationRecommender(URM_train, [recommender_item, recommender_rp3, recommender_p3a], weights)
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

print('Linear Combinator (Item knn, p3alpha, rp3)')
result_df, _ = evaluator.evaluateRecommender(linear_combinator)
display(result_df)
print('Merging Models')
result_df, _ = evaluator.evaluateRecommender(hybrid_similarity)
display(result_df)
print('Item KNN')
result_df, _ = evaluator.evaluateRecommender(recommender_item)
display(result_df)
print('RP3 beta')
result_df, _ = evaluator.evaluateRecommender(recommender_rp3)
display(result_df)
print('P3 alpha')
result_df, _ = evaluator.evaluateRecommender(recommender_p3a)
display(result_df)
print('User KNN')
result_df, _ = evaluator.evaluateRecommender(recommender_user)
display(result_df)
print('NMF')
result_df, _ = evaluator.evaluateRecommender(recommender_nmf)
display(result_df)
print('Pure SVD')
result_df, _ = evaluator.evaluateRecommender(recommender_pure_svd)
display(result_df)
print('Item Pure SVD')
result_df, _ = evaluator.evaluateRecommender(recommender_pure_svd_item)
display(result_df)
print('SLIM BPR')
result_df, _ = evaluator.evaluateRecommender(recommender_slim_bpr)
display(result_df)
print('SLIM Elastic')
result_df, _ = evaluator.evaluateRecommender(recommender_slim)
display(result_df)
print('EASE R')
result_df, _ = evaluator.evaluateRecommender(recommender_ease_r)
display(result_df)
print('IALS')
result_df, _ = evaluator.evaluateRecommender(recommender_ials)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
Linear Combinator (Item knn, p3alpha, rp3)
EvaluatorHoldout: Processed 10456 (100.0%) in 17.75 sec. Users per second: 589


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.096815,0.165245,0.139023,0.050068,0.085341,0.274007,0.152473,0.114142,0.518363,0.365435,...,0.802826,0.416155,0.802826,0.081667,10.62976,0.99837,0.235642,0.818875,1.537517,0.308264


Merging Models
EvaluatorHoldout: Processed 10456 (100.0%) in 9.14 sec. Users per second: 1144


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.097102,0.165756,0.139351,0.050173,0.085482,0.27428,0.152762,0.114452,0.520658,0.365828,...,0.802826,0.417998,0.802826,0.087463,10.729146,0.998493,0.252365,0.826532,1.477653,0.310478


Item KNN
EvaluatorHoldout: Processed 10456 (100.0%) in 9.43 sec. Users per second: 1109


...

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.097379,0.16502,0.138508,0.051089,0.086077,0.282425,0.154158,0.114358,0.524962,0.375002,...,0.802826,0.421453,0.802826,0.065098,10.631702,0.9989,0.187832,0.819025,1.417992,0.303398


EASE R
EvaluatorHoldout: Processed 10456 (100.0%) in 15.78 sec. Users per second: 663


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.074857,0.135323,0.116287,0.037511,0.067475,0.233553,0.123573,0.091082,0.459353,0.293884,...,0.802826,0.368781,0.802826,0.019069,8.350538,0.991242,0.055022,0.643293,3.089786,0.267475


IALS
EvaluatorHoldout: Processed 10456 (100.0%) in 14.96 sec. Users per second: 699


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.08224,0.144123,0.122645,0.041501,0.071584,0.246255,0.131701,0.098458,0.479055,0.316769,...,0.802826,0.384598,0.802826,0.029074,9.74689,0.998532,0.08389,0.750862,1.599865,0.289534


In [23]:
weights = [0.8, 0.4, 0.7, 0.6, 0.2, 0.05, 0.01, 0.006, 0.008, 0.003]
recommenders = [recommender_slim, recommender_item, recommender_rp3, recommender_p3a, recommender_ials, recommender_ease_r, recommender_user, recommender_pure_svd_item, recommender_nmf, recommender_slim_bpr]
linear_combinator = LinearCombinationRecommender(URM_train, recommenders, weights)
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])
result_df, _ = evaluator.evaluateRecommender(linear_combinator)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 51.59 sec. Users per second: 203


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.099857,0.172291,0.145385,0.051895,0.088519,0.284224,0.158167,0.118395,0.534239,0.379028,...,0.802826,0.4289,0.802826,0.062043,10.355214,0.998148,0.179019,0.797725,1.697002,0.299734


In [24]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

def objective(trial):
    weights = [
        trial.suggest_float('slim_elastic', 1e-2, 1, log=True),
        trial.suggest_float('item_knn', 1e-2, 1, log=True),
        trial.suggest_float('rp3', 1e-2, 1, log=True),
        trial.suggest_float('p3alpha', 1e-2, 1, log=True),
        trial.suggest_float('ials', 5e-3, 1, log=True),
        trial.suggest_float('ease_r', 1e-3, 1, log=True),
        trial.suggest_float('user_knn', 1e-3, 1, log=True),
        trial.suggest_float('svd_item', 1e-4, 0.2, log=True),
        trial.suggest_float('nmf', 1e-4, 0.2, log=True),
        trial.suggest_float('slim_bpr', 1e-5, 0.1, log=True),
    ]
    
    recommenders = [
        recommender_slim, recommender_item, recommender_rp3, recommender_p3a, recommender_ials, 
        recommender_ease_r, recommender_user, recommender_pure_svd_item, recommender_nmf, recommender_slim_bpr
    ]
    
    linear_combinator = LinearCombinationRecommender(URM_train, recommenders, weights)
    evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])
    result_df, _ = evaluator.evaluateRecommender(linear_combinator)

    return result_df.loc[10, 'MAP']

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions


In [25]:
# def evaluate_linear_model(URM_train, URM_valid, weights):
#     recommender_item = ItemKNNCFRecommender(URM_train)
#     recommender_item.fit(**params_item_knn)

#     recommender_rp3 = RP3betaRecommender(URM_train)
#     recommender_rp3.fit(**params_rp3)
    
#     recommender_p3a = P3alphaRecommender(URM_train)
#     recommender_p3a.fit(**params_p3alpha)
    
#     linear_combinator = LinearCombinationRecommender(URM_train, [recommender_item, recommender_rp3, recommender_p3a], weights)    
#     evaluator = EvaluatorHoldout(URM_valid, cutoff_list=[10])
#     result_df, _ = evaluator.evaluateRecommender(linear_combinator)
#     return result_df.loc[10, 'MAP']

# def objective(trial):
#     w1 = trial.suggest_float('w1', 0.01, 0.99)
#     w2 = trial.suggest_float('w2', 0.01, 0.99)
#     w3 = trial.suggest_float('w3', 0.01, 0.99)
    
#     results = [None] * 10
#     with ThreadPoolExecutor(max_workers=10) as executor:
#         futures = []
#         for URM_train, URM_valid in CrossValidationSplitter(10).split(URM_all):
#             futures.append(
#                 executor.submit(evaluate_linear_model, URM_train, URM_valid, [w1, w2, w3])
#             )
#         for i, future in enumerate(futures):
#             results[i] = future.result()
#     return np.mean(results)

In [26]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_hybrid_merging.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass

In [27]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_linear_10models_no_cv_v2', 
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective, n_trials=10)

[I 2023-12-03 21:12:51,319] Using an existing study with name 'hyperparameters_tuning_linear_10models_no_cv_v2' instead of creating a new one.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 51.33 sec. Users per second: 204


[I 2023-12-03 21:13:43,408] Trial 400 finished with value: 0.05092922268371834 and parameters: {'slim_elastic': 0.09720107654382212, 'item_knn': 0.05353935310773328, 'rp3': 0.4119338809438938, 'p3alpha': 0.3395565390656278, 'ials': 0.011260318597291116, 'ease_r': 0.0020506152129117403, 'user_knn': 0.009333004808818178, 'svd_item': 0.008768752213467627, 'nmf': 0.0013168792903905165, 'slim_bpr': 0.0001952294267793505}. Best is trial 174 with value: 0.05233878778494365.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 52.66 sec. Users per second: 199


[I 2023-12-03 21:14:36,456] Trial 401 finished with value: 0.05223001375378044 and parameters: {'slim_elastic': 0.7358167562243899, 'item_knn': 0.042539160780342214, 'rp3': 0.14100566364184183, 'p3alpha': 0.49363546646034573, 'ials': 0.02913586706730206, 'ease_r': 0.0027953569171079687, 'user_knn': 0.008418378414800733, 'svd_item': 0.015245332484678567, 'nmf': 0.001689409840096218, 'slim_bpr': 0.0003021060518085739}. Best is trial 174 with value: 0.05233878778494365.


...

[I 2023-12-03 21:19:59,641] Trial 407 finished with value: 0.05200364945774329 and parameters: {'slim_elastic': 0.6733198736068386, 'item_knn': 0.02523208586699759, 'rp3': 0.14821284126999038, 'p3alpha': 0.6989738541462643, 'ials': 0.01027331724223152, 'ease_r': 0.0016657833046082645, 'user_knn': 0.002541487549321528, 'svd_item': 0.016000257406897853, 'nmf': 0.0011737666436702513, 'slim_bpr': 0.001174652574329502}. Best is trial 174 with value: 0.05233878778494365.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 52.75 sec. Users per second: 198


[I 2023-12-03 21:20:52,778] Trial 408 finished with value: 0.050704585200569106 and parameters: {'slim_elastic': 0.3231314541438313, 'item_knn': 0.8769537273553407, 'rp3': 0.4761285842778945, 'p3alpha': 0.21650131726225222, 'ials': 0.0114667380725597, 'ease_r': 0.0014369887500357355, 'user_knn': 0.016365980464509145, 'svd_item': 0.0003637255624346563, 'nmf': 0.0018127891581630174, 'slim_bpr': 0.00024383005042996676}. Best is trial 174 with value: 0.05233878778494365.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 51.97 sec. Users per second: 201


[I 2023-12-03 21:21:45,135] Trial 409 finished with value: 0.049946392896613546 and parameters: {'slim_elastic': 0.010948277338456932, 'item_knn': 0.046763197437412224, 'rp3': 0.1624213576907502, 'p3alpha': 0.5567140559958842, 'ials': 0.009400573714622938, 'ease_r': 0.004261367447186267, 'user_knn': 0.0056708723853718, 'svd_item': 0.011810720411949067, 'nmf': 0.0013843249427209457, 'slim_bpr': 1.4812588513703151e-05}. Best is trial 174 with value: 0.05233878778494365.


In [28]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [29]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [30]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [31]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_hybrid_lr_comb_10models.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_hybrid_lr_comb_10models.json', 
            'tuning_results/best_params_hybrid_lr_comb_10models.json', 
            'Linear combination of 10 models tuning results (from kaggle notebook)'
        )
    
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/input/linear-combination/best_params_hybrid_lr_comb_10models.json', 
        '/kaggle/working/best_params_hybrid_lr_comb_10models.json'
    )

In [32]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'],
        'tuning_results/tuning_hybrid_merging.db', 
        'Linear combination tuning db updated results (from kaggle notebook)'
    )

In [33]:
with open('/kaggle/working/best_params_hybrid_lr_comb_10models.json', 'r') as params_file:
    params = json.load(params_file)
params

{'slim_elastic': 0.7055016257785176,
 'item_knn': 0.04567821515549036,
 'rp3': 0.2128594852139593,
 'p3alpha': 0.48026637063438987,
 'ials': 0.011047513804993822,
 'ease_r': 0.009813389141426643,
 'user_knn': 0.011492873259506146,
 'svd_item': 0.009253225835051558,
 'nmf': 0.0011546638940866953,
 'slim_bpr': 0.0006458494215963735}

In [34]:
recommender_item = ItemKNNCFRecommender(URM_all)
recommender_item.fit(**params_item_knn)

recommender_user = UserKNNCFRecommender(URM_all)
recommender_user.fit(**params_user_knn)

recommender_rp3 = RP3betaRecommender(URM_all)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_all)
recommender_p3a.fit(**params_p3alpha)

recommender_pure_svd = PureSVDRecommender(URM_all)
recommender_pure_svd.fit(num_factors=params_pure_svd['factors'])

recommender_pure_svd_item = PureSVDItemRecommender(URM_all)
recommender_pure_svd_item.fit(**params_pure_svd_item)

recommender_nmf = NMFRecommender(URM_all)
recommender_nmf.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'nmf_all.zip')

recommender_slim_bpr = SLIM_BPR_Python(URM_all)
recommender_slim_bpr.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_bpr_all.zip')

recommender_slim = SLIMElasticNetRecommender(URM_all)
recommender_slim.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_elastic_all.zip')

recommender_ease_r = EASE_R_Recommender(URM_all)
recommender_ease_r.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'ease_r_all.zip')

recommender_ials = IALSRecommender(URM_all)
recommender_ials.load_model('/kaggle/working/', 'ials_all.zip')

ItemKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 22347 (100.0%), 4192.52 column/sec. Elapsed time 5.33 sec
UserKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
UserKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 13024 (100.0%), 3911.98 column/sec. Elapsed time 3.33 sec
RP3betaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2307.47 column/sec. Elapsed time 9.68 sec
P3alphaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
P3alphaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 2563.12 column/sec. Elapsed time 8.72 sec
PureSVDRecommender: URM Detected 386 ( 3.0%) users with no inter

In [35]:
weights = [
    params['slim_elastic'],
    params['item_knn'],
    params['rp3'],
    params['p3alpha'],
    params['ials'],
    params['ease_r'],
    params['user_knn'],
    params['svd_item'],
    params['nmf'],
    params['slim_bpr']
]

recommenders = [
    recommender_slim, recommender_item, recommender_rp3, recommender_p3a, recommender_ials, 
    recommender_ease_r, recommender_user, recommender_pure_svd_item, recommender_nmf, recommender_slim_bpr
]

In [36]:
linear_combinator = LinearCombinationRecommender(URM_all, recommenders, weights)
recommendations = linear_combinator.recommend(user_ids_test['user_id'].unique() - 1, cutoff=10, return_scores=False)
recommendations = [' '.join([str(item + 1) for item in r]).strip() for r in recommendations]

submission_df = pd.DataFrame({'user_id': user_ids_test['user_id'].unique(), 'item_list': recommendations})
display(submission_df.head())

submission_df.to_csv('/kaggle/working/submission_linear_comb_10models.csv', index=False)

Recommender_Base_Class: URM Detected 386 ( 3.0%) users with no interactions.
Recommender_Base_Class: URM Detected 125 ( 0.6%) items with no interactions.


,user_id,item_list
0,1,101 36 123 506 403 515 694 1546 52 1422
1,2,1095 47 12 28 50 1522 656 196 102 11
2,3,59 259 857 4252 536 648 584 2172 956 4
3,4,28 249 50 139 171 314 7 254 146 5
4,5,1570 77 170 95 131 1511 5138 471 1220 116
